In [65]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [66]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [67]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [68]:
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]: 
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] #crop and append
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [69]:
X.shape, Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [70]:
C = torch.randn((27,2))

In [71]:
C[5]

tensor([1.0893, 0.0905])

In [72]:
F.one_hot(torch.tensor(5),num_classes = 27).float() @ C #from previous lecture - same as above  

tensor([1.0893, 0.0905])

In [73]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [74]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)


In [75]:
#emb @ W1 +b1    # error because vectors' dimentions are not compatible

In [76]:
torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]],1).shape

torch.Size([32, 6])

In [77]:
torch.cat(torch.unbind(emb,1),1).shape # same as above

torch.Size([32, 6])

In [78]:
emb.view(32,6).shape # same as above but that is faster than others(also better in terms of memory) so we use this

torch.Size([32, 6])

In [79]:
h = torch.tanh(emb.view(-1,6) @ W1 +b1)
h

tensor([[-1.0000, -0.9996, -1.0000,  ..., -0.9999, -1.0000,  0.9817],
        [-0.9985, -0.9999, -0.9697,  ...,  0.8651,  0.9997,  0.4805],
        [ 0.9617, -0.9988, -1.0000,  ...,  0.9998, -0.9954,  0.9972],
        ...,
        [ 0.9996, -0.9998, -0.9436,  ...,  0.9976,  0.9735,  0.9431],
        [ 1.0000,  0.2580, -0.7371,  ...,  0.9982, -0.8761,  0.8196],
        [ 0.9937,  0.9998,  0.7240,  ...,  0.9878,  0.9620, -0.8778]])

In [80]:
h.shape

torch.Size([32, 100])

In [81]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [82]:
logits = h @ W2 +b2

In [83]:
logits.shape

torch.Size([32, 27])

In [84]:
counts = logits.exp()
prob = counts / counts.sum(1,keepdims= True)

In [85]:
prob.shape

torch.Size([32, 27])

In [86]:
loss = -prob[torch.arange(32),Y].log().mean()
loss

tensor(16.0091)

In [87]:
# ------------- edited version --------------

In [88]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [92]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [93]:
sum(p.nelement() for p in parameters) #number of parameters in total

3481

In [95]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
counts = logits.exp()
prob = counts / counts.sum(1, keepdims = True)
loss = -prob[torch.arange(32),Y].log().mean()
loss

tensor(17.7697)